In [ ]:
%cd ~
import matplotlib
matplotlib.use("Agg")
import pyfits,glob,time,scipy
import scipy.interpolate
from scipy.special import erf
from pylab import *
from subprocess import call
from IPython import parallel
from scipy.optimize import fmin_powell

topdir='/sandbox/lsst/lsst/GUI/'
thedir=topdir+'notebooks/'
datadir=topdir+'20161012-cte/'
%cd $thedir

configfile=topdir+'sextractor/default-array_dither.sex'
paramfile=topdir+'sextractor/default-array_dither.param'
maskdir=topdir+'sextractor/masks/'


In [ ]:
def remove_overscan_xy(image,x_overscan_start,x_overscan_end,y_overscan_start,y_overscan_end):
    overscan=image[:y_overscan_start,x_overscan_start+1:x_overscan_end]
    image=image[:y_overscan_start,:x_overscan_start]
    finalimg=image-matrix(median(overscan,axis=1)).T*np.ones(shape(image)[1])
    return array(finalimg)

def make_reg_from_ldac(cat_ldac_file,text_tag):
    thecat=pyfits.getdata(cat_ldac_file,'LDAC_OBJECTS')
    f = open(cat_ldac_file+'.reg','w')
    for i in range(len(thecat)):
        xcoo,ycoo=thecat['XWIN_IMAGE'][i],thecat['YWIN_IMAGE'][i]
        r=thecat['A_IMAGE'][i]
        thetext=thecat[text_tag][i]
        f.write('circle '+str(xcoo)+' '+str(ycoo)+' '+str(r)+'#text="'+str(thetext)+'"\n')
    f.close()
    
def Area(xl, xh, yl, yh, sigmax, sigmay, Imax):
    # Calculates how much of a 2D Gaussian falls within a rectangular box
    ssigx = sqrt(2) * sigmax
    ssigy = sqrt(2) * sigmay    
    I = (erf(xh/ssigx)-erf(xl/ssigx))*(erf(yh/ssigy)-erf(yl/ssigy))
    return Imax * I / 4.0

In [ ]:
# This definition runs sextractor in parallel and can be given to IPython's parallel map function along with
#   the file list
def ovsub_runsex_makereg(fitsfilename):
    import pyfits
    import numpy as np
    from subprocess import call
    topdir='/Users/cslage/Research/LSST/code/GUI/'
    thedir=topdir+'profiles'
    %cd $thedir

    configfile=topdir+'sextractor/default-array_dither.sex'
    paramfile=topdir+'sextractor/default-array_dither.param'
    maskdir=topdir+'sextractor/masks/'

    def remove_overscan_xy(image,x_overscan_start,x_overscan_end,y_overscan_start,y_overscan_end):
        overscan=image[:y_overscan_start,x_overscan_start+1:x_overscan_end]
        image=image[:y_overscan_start,:x_overscan_start]
        finalimg=image-np.matrix(np.median(overscan,axis=1)).T*np.ones(np.shape(image)[1])
        return finalimg

    def make_reg_from_ldac(cat_ldac_file,text_tag):
        thecat=pyfits.getdata(cat_ldac_file,'LDAC_OBJECTS')
        f = open(cat_ldac_file+'.reg','w')
        for i in range(len(thecat)):
            xcoo,ycoo=thecat['XWIN_IMAGE'][i],thecat['YWIN_IMAGE'][i]
            r=thecat['A_IMAGE'][i]
            thetext=thecat[text_tag][i]
            f.write('circle '+str(xcoo)+' '+str(ycoo)+' '+str(r)+'#text="'+str(thetext)+'"\n')
        f.close()
    for i in range(1,17):
        extname=pyfits.getheader(fitsfilename,i)['EXTNAME']
        img=pyfits.getdata(fitsfilename,extname)
        overscansubimg=remove_overscan_xy(img,509,542,2000,2022)   # cut off the overscan
        outname=fitsfilename[:-5]+extname+'.fits'
        pyfits.writeto(outname,overscansubimg,clobber=True)
        test=call(["sex",outname,"-c",configfile,"-CATALOG_NAME",outname+'.cat'])
        make_reg_from_ldac(outname+'.cat','NUMBER')

In [ ]:
zfilelist=sort(glob.glob(datadir+'ITL-3800C-029_spot-30um_light_130_20161012??????_bias.fits'))
print len(zfilelist)
zfilelist

In [ ]:
for fitsfilename in zfilelist: 
    tfile1=time.time() 
    for i in [4]:
        tstart=time.time() 
        extname=pyfits.getheader(fitsfilename,i)['EXTNAME'] 
        img=pyfits.getdata(fitsfilename,extname) 
        overscansubimg=remove_overscan_xy(img,509,542,2000,2022) 
        # cut off the overscan 
        outname=fitsfilename[:-5]+extname+'.fits' 
        pyfits.writeto(outname,overscansubimg,clobber=True) 
        test=call(["sex",outname,"-c",configfile,"-CATALOG_NAME",outname+'.cat']) 
        make_reg_from_ldac(outname+'.cat','NUMBER') 
        tend=time.time() 
        print extname+" time: "+str(tend-tstart)[:4] 
        print "Time taken for file "+str(fitsfilename[-26:-23])+": "+str(time.time()-tfile1)

#### Using the sextractor catalogs produced above to make a map of the sextractor measurement named below

In [ ]:
# [seg, xstart, xpeak, ystart, ystop]
regions = [[3,130,148.311,1820,1821]]
figure()
subplots_adjust(hspace = 0.5, wspace = 0.5)
suptitle("CTI - Spot", fontsize = 18)
plotcounter = 1
for file in zfilelist:
    hdulist = pyfits.open(file, mode='readonly', do_not_scale_image_data=True)
    for [segment,xstart,xpeak,ystart,ystop] in regions:
        xstop = xstart+40
        xaxis = linspace(xstart,xstop-1,40)
        subplot(1,1,plotcounter)
        plotcounter += 1
        imhdr=hdulist[segment+1].header
        extname = imhdr['EXTNAME']
        title("%s, x = %d, y = %d"%(extname,int(xpeak),(int((ystart+ystop)/2))), fontsize = 10)
        data = array(hdulist[segment+1].data + 32768, dtype = int32)
        cte_data = data[ystart:ystop,:].mean(axis=0)[xstart:xstop]
        #cte_std = (data[ystart:ystop,:].std(axis=0) / sqrt(float(ystop-ystart)))[xstart:xstop]
        #ratio = data[ystart:ystop,xpeak+1].mean() / data[ystart:ystop,xpeak].mean()
        #cti = ratio / xpeak
        scatter(xaxis, cte_data, color = 'blue')
        #plot([xpeak,xpeak],[1.0,100000],ls = '--')
        #errorbar(xaxis, cte_data, yerr=cte_std)
        #text(xstart-2, 20000, "Ratio = %.3E"%ratio,fontsize=6)
        #text(xstart-2, 10000, "CTI = %.3E"%cti,fontsize=6)
        text(xstart,100,"<----",color = 'blue')
        yscale('log',nonposy='clip')
        ylim(1.0, 100000)
        xlabel("X")
        ylabel("Signal(ADU)")
savefig(datadir+"Serial_CTE_Spot_12Oct16.pdf")
